# Add Liquidity
Position $\alpha$ adds liquidity with $\Delta r_\alpha$ of asset $i$.

### Constraints
Adding liquidity must leave prices $p_j^Q$ unchanged for all assets $j$. It must also leave $\frac{R_i}{S_i}$ unchanged.

### Requirements
$$
\begin{align}
s_\alpha &= 0\\
r_\alpha &\geq -\Delta r_\alpha\\
\Delta r_\alpha &< 0
\end{align}
$$

### Updating AMM state
$$
\Delta Q_i = Q_i \frac{\Delta R_i}{R_i}\\
$$
If $\frac{Q_i + \Delta Q_i}{Q} > \omega_i$, the transaction fails due to the weight cap on asset $i$.

$$
\begin{align}
\Delta S_i &= S_i \frac{\Delta R_i}{R_i}\\
\Delta R_i &= -\Delta r_\alpha\\
\Delta L &= \Delta R_i \frac{Q_i}{R_i} \frac{L}{Q}
\end{align}
$$

Let $U$ be the asset index of the selected stablecoin in Omnipool.
$$
\begin{align}
\Delta T_i &= Q_i^+ \frac{R_U^+}{Q_U^+} - T_i\\
\Delta T &= \Delta T_i
\end{align}
$$

If $T + \Delta T > C$, the transaction fails due to violation of the overall TVL cap $C$.

### Updating agent state
$$
\begin{align}
\Delta s_\alpha &= \Delta S_i\\
p_\alpha &= p_i^Q
\end{align}
$$

## Checking the algebra


In [1]:
from sympy.solvers import solve
from sympy import Symbol
R = Symbol('R')
Q = Symbol('Q')
S = Symbol('S')
Qall = Symbol('Qall')
L = Symbol('L')

dR = Symbol('dR')

dQ = Q * dR/R
dS = S * dR/R
dL = dR * Q/R * L/Qall

Our claims are that
- $\frac{R_i}{S_i} = \frac{R_i + \Delta R_i}{S_i + \Delta S_i}$
- $\frac{Q_i}{R_i} = \frac{Q_i + \Delta Q_i}{R_i + \Delta R_i}$, i.e. price is unchanged
- $\frac{Q_i}{R_i}\frac{Q + L}{Q} = \frac{Q_i + \Delta Q_i}{R_i + \Delta R_i}\frac{Q + \Delta Q_i + L + \Delta L}{Q + \Delta Q_i}$, i.e. target price is unchanged

In [2]:
# We are given dR by user
# S, Q and R are starting state of AMM

print("dS_i")
print(solve(R / S - (R + dR)/(S + dS), dS))
print("dQ_i")
print(solve(Q / R - (Q + dQ)/(R + dR), dQ))

print("target price")
lhs = Q/R * (Qall + L)/Qall
rhs = (Q + dQ)/(R + dR) * (Qall + dQ + L + dL)/(Qall + dQ)
print(lhs.equals(rhs))

dS_i
[S*dR/R]
dQ_i
[Q*dR/R]
Q/R
True
